# Lab08
- Brian Carrillo
- Josué Morales
- Marco Ramírez
- Carlos López

### Ejercicio 03

#### Genetic Algorithm

In [16]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import pandas as pd
import gzip
import io
from tqdm import tqdm

def read_tsp_file(file_path):
    coords = []
    with gzip.open(file_path, 'rt') as f:
        lines = f.readlines()
        start_coords = False
        for line in lines:
            if line.strip() == "NODE_COORD_SECTION":
                start_coords = True
                continue
            if line.strip() == "EOF":
                break
            if start_coords:
                _, x, y = line.strip().split()
                coords.append([float(x), float(y)])
    return np.array(coords)

def calculate_distance(route, coords):
    total_distance = 0
    for i in range(len(route)):
        city1 = coords[route[i]]
        city2 = coords[route[(i + 1) % len(route)]]
        total_distance += np.sqrt(np.sum((city1 - city2) ** 2))
    return total_distance

def create_initial_population(pop_size, num_cities):
    population = []
    for _ in range(pop_size):
        route = np.random.permutation(num_cities)
        population.append(route)
    return population

def tournament_selection(population, distances, tournament_size=5):
    tournament = np.random.choice(len(population), tournament_size, replace=False)
    tournament_distances = [distances[i] for i in tournament]
    winner_idx = tournament[np.argmin(tournament_distances)]
    return population[winner_idx]

def order_crossover(parent1, parent2):
    size = len(parent1)
    start, end = sorted(np.random.randint(0, size, 2))
    
    offspring = np.full(size, -1)
    offspring[start:end] = parent1[start:end]
    
    remaining_cities = [city for city in parent2 if city not in offspring[start:end]]
    j = 0
    for i in range(size):
        if offspring[i] == -1:
            offspring[i] = remaining_cities[j]
            j += 1
            
    return offspring

def swap_mutation(route, mutation_rate=0.01):
    for i in range(len(route)):
        if np.random.random() < mutation_rate:
            j = np.random.randint(0, len(route))
            route[i], route[j] = route[j], route[i]
    return route

def genetic_algorithm(coords, pop_size=100, generations=100000, mutation_rate=0.01, 
                     survival_rate=0.1, patience=100000, tournament_size=5):
    num_cities = len(coords)
    population = create_initial_population(pop_size, num_cities)
    
    best_distances = []
    avg_distances = []
    survival_rates = []
    
    best_ever_distance = float('inf')
    best_ever_route = None
    no_improvement = 0
    
    for gen in tqdm(range(generations), desc="Generations"):
        # Evaluate fitness
        distances = [calculate_distance(route, coords) for route in population]
        
        # Sort population by fitness
        sorted_indices = np.argsort(distances)
        current_best_distance = distances[sorted_indices[0]]
        current_best_route = population[sorted_indices[0]]
        
        # Update best ever solution
        if current_best_distance < best_ever_distance:
            best_ever_distance = current_best_distance
            best_ever_route = current_best_route.copy()
            no_improvement = 0
        else:
            no_improvement += 1
        
        # Store statistics
        best_distances.append(current_best_distance)
        avg_distances.append(np.mean(distances))
        survival_rates.append(len(set(map(tuple, population))) / pop_size)
        
        # Early stopping
        if no_improvement >= patience:
            print(f"\nEarly stopping at generation {gen} - No improvement for {patience} generations")
            break
        
        # Create new population
        new_population = []
        
        # Elitism: keep the best routes
        num_survivors = int(pop_size * survival_rate)
        new_population.extend([population[i] for i in sorted_indices[:num_survivors]])
        
        # Create rest of new population
        while len(new_population) < pop_size:
            # Selection
            parent1 = tournament_selection(population, distances, tournament_size)
            parent2 = tournament_selection(population, distances, tournament_size)
            
            # Crossover
            offspring = order_crossover(parent1, parent2)
            
            # Mutation
            offspring = swap_mutation(offspring, mutation_rate)
            
            new_population.append(offspring)
        
        population = new_population
    
    return {
        'best_distances': best_distances,
        'avg_distances': avg_distances,
        'survival_rates': survival_rates,
        'best_ever_route': best_ever_route,
        'best_ever_distance': best_ever_distance
    }

def plot_final_route(coords, best_route):
    plt.figure(figsize=(10, 10))
    
    # Plot route
    route_coords = np.vstack((coords[best_route], coords[best_route[0]]))  # Add first city at end
    plt.plot(route_coords[:, 0], route_coords[:, 1], 'b-', linewidth=1)
    plt.scatter(coords[:, 0], coords[:, 1], c='red', s=50)
    
    plt.title('Final Route')
    plt.savefig('final_route.png')
    plt.close()

def plot_statistics(results):
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
    
    # Plot distances
    generations = range(len(results['best_distances']))
    ax1.plot(generations, results['best_distances'], label='Best Distance')
    ax1.plot(generations, results['avg_distances'], label='Average Distance')
    ax1.set_xlabel('Generation')
    ax1.set_ylabel('Distance')
    ax1.set_title('Evolution of Distances')
    ax1.legend()
    ax1.grid(True)
    
    # Plot survival rate
    ax2.plot(generations, results['survival_rates'], label='Diversity Rate')
    ax2.set_xlabel('Generation')
    ax2.set_ylabel('Population Diversity')
    ax2.set_title('Population Diversity Over Time')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.savefig('statistics.png')
    plt.close()

# Read TSP file
coords = read_tsp_file('ch150.tsp.gz')

# Run genetic algorithm
results = genetic_algorithm(coords)

# Plot final route
plot_final_route(coords, results['best_ever_route'])

# Plot statistics
plot_statistics(results)

# Print optimal tour and distance
print(f"Optimal distance: {results['best_ever_distance']}")

Generations: 100%|██████████| 100000/100000 [5:50:33<00:00,  4.75it/s]      


Optimal distance: 8950.854148251197
